# Dinámica de la fase 2 según el destino de la transición

> Hipótesis de trabajo: *"La fase 2 (S2) no es estática; la duración del episodio refleja la preparación del cerebro para la siguiente transición."*

## Narrativa

1. La secuencia de etapas de sueño se interpreta como texto: cada fase equivale a un símbolo que puede alimentarse a Word2Vec.
2. Simplificamos el alfabeto para reducir ruido: **S3=S4** y **Movimiento=Sin clasificar**.
3. Construimos dos métricas para cada transición que abandona S2:
   - **Tiempo de espera en S2**: cuánto dura el bloque completo de S2 justo antes de saltar.
   - **Tiempo de llegada**: cuánto dura la fase destino justo después de abandonar S2.
4. Contrastamos ambas distribuciones con tablas y boxplots para detectar si ciertos destinos se asocian a preparaciones (S2) más largas o más cortas.
5. Estas señales alimentarán los embeddings Skip-Gram para validar si los vectores capturan la dirección de la transición.

## Plan de acción

1. Importar utilidades, rutas y definir el nuevo alfabeto compacto de fases.
2. Cargar todos los hipnogramas **scor_clean** (sin remapear) y conservar la secuencia cruda; el mapeo al alfabeto reducido sólo se aplica al destino cuando resumimos resultados.
3. Extraer cada bloque continuo de S2, identificando su origen, destino, duración y la longitud de la fase destino inmediatamente posterior.
4. Construir tablas descriptivas:
   - Longitudes de espera en S2 por paciente y de forma global.
   - Duraciones del destino y métricas emparejadas **espera vs llegada** para cada tipo de transición **2 → X**.
5. Generar boxplots con fondo oscuro para comparar (a) la espera en S2 y (b) la duración de la fase destino, tanto por paciente como global.
6. Exportar los diccionarios resultantes para reutilizarlos en el notebook de embeddings.

In [25]:
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display
from scipy import stats


In [26]:
RUTA_DATOS = Path("../Datos")

ETIQUETAS_FASES = {
    0: "Vigilia (W)",
    1: "S1",
    2: "S2",
    3: "S3/S4",
    4: "REM",
    5: "Movimiento/SC"
}

ORDEN_DESTINOS = [ETIQUETAS_FASES[i] for i in sorted(ETIQUETAS_FASES.keys())]

pd.options.display.float_format = lambda x: f"{x:0.3f}"


def agrupar_fase(valor):
    """Mapea cualquier fase cruda al alfabeto reducido."""
    return {4: 3, 5: 4, 6: 5, 7: 5}.get(valor, valor)


def remapear_secuencia(secuencia):
    """Aplica `agrupar_fase` a cada época para unificar símbolos antes del conteo."""
    return [agrupar_fase(valor) for valor in secuencia]


In [27]:
def listar_pacientes(ruta_datos=None):
    """Devuelve los IDs disponibles (archivos `Scoring_*_2`)."""
    ruta_datos = ruta_datos or RUTA_DATOS
    pacientes = []
    for archivo in ruta_datos.iterdir():
        nombre = archivo.name
        if archivo.is_file() and nombre.startswith("Scoring_") and nombre.endswith("_2"):
            pacientes.append(nombre.replace("Scoring_", ""))
    pacientes.sort()
    if not pacientes:
        raise RuntimeError("No se encontraron archivos `Scoring_*_2` en la ruta indicada.")
    return pacientes


def cargar_hipnograma(paciente, ruta_datos=None):
    """Carga la columna `scor_clean` y la devuelve como lista de enteros."""
    ruta_datos = ruta_datos or RUTA_DATOS
    ruta_archivo = ruta_datos / f"Scoring_{paciente}"
    columnas = ["epoca", "scor_clean", "scor_quasi"]
    df = pd.read_csv(ruta_archivo, sep=r"\s+", header=None, names=columnas, dtype=int)
    return df["scor_clean"].tolist()


In [28]:
pacientes = listar_pacientes()
print(f"Pacientes detectados ({len(pacientes)}): {', '.join(pacientes)}")

# CORRECCIÓN: No aplicamos remapear_secuencia aquí.
# Guardamos los datos crudos para medir la duración exacta de la fase específica (ej. solo S3 o solo S4).
hipnogramas = {}
for paciente in pacientes:
    # Cargar crudo (0-7)
    crudo = cargar_hipnograma(paciente)
    hipnogramas[paciente] = crudo  # <--- CAMBIO: Usamos crudo directamente

print("\nAlfabeto usado para el conteo (Original):")
print("  0: W, 1: N1, 2: N2, 3: N3, 4: N4, 5: REM, 6: Mov, 7: SC")
print("\nAlfabeto agrupado para visualización:")
for codigo, etiqueta in ETIQUETAS_FASES.items():
    print(f"  {codigo}: {etiqueta}")


Pacientes detectados (10): AR_2, DG_2, EL_2, GA_2, IN_2, JS_2, LL_2, SS_2, VB_2, VC_2

Alfabeto usado para el conteo (Original):
  0: W, 1: N1, 2: N2, 3: N3, 4: N4, 5: REM, 6: Mov, 7: SC

Alfabeto agrupado para visualización:
  0: Vigilia (W)
  1: S1
  2: S2
  3: S3/S4
  4: REM
  5: Movimiento/SC


## Extracción de bloques S2 → destino

Cada bloque continuo de S2 se describe con:
- **origen**: fase inmediatamente anterior a la entrada en S2 (puede ser **None** si el registro inicia en S2).
- **destino**: fase inmediatamente posterior al bloque de S2 (ya remapeado; S3 y S4 son un mismo símbolo, al igual que Movimiento/Sin clasificar).
- **duracion_s2**: número de épocas consecutivas en S2 (tiempo de espera).
- **duracion_destino**: número de épocas consecutivas en la fase destino (tiempo de llegada) considerando la secuencia ya remapeada, por lo que bloques S3+S4 se suman correctamente.

Ejemplo: para la secuencia remapeada **22223344** (originalmente 2222-3-3-4-4), la espera es 4 (cuatro épocas consecutivas en S2: **2222**) y la llegada es 4 (las cuatro épocas que representan S3/S4: **3344** → **3333** tras el remapeo).

Esto permite comparar cuánto “se prepara” S2 dependiendo del lugar al que transita.

In [29]:
def extraer_bloques_s2(secuencia):
    """
    Devuelve una lista de dicts con metadatos por bloque continuo de S2.
    
    CORRECCIÓN METODOLÓGICA: Trabaja con la secuencia cruda (valores 0-7) para calcular
    duraciones exactas. La agrupación (S3/S4, Mov/SC) se aplica solo a las etiquetas
    de visualización, no a las duraciones.
    
    Cada bloque contiene:
    - origen: fase anterior (cruda, puede ser None)
    - destino_raw: fase destino en formato crudo (0-7)
    - duracion_s2: duración del bloque S2
    - duracion_destino: duración del bloque destino (calculada sobre el valor crudo)
    """
    bloques = []
    indice = 0
    total = len(secuencia)
    while indice < total:
        if secuencia[indice] != 2:
            indice += 1
            continue
        inicio = indice
        origen = secuencia[indice - 1] if indice > 0 else None
        while indice < total and secuencia[indice] == 2:
            indice += 1
        fin = indice  # primer índice != 2
        duracion_s2 = fin - inicio
        if fin >= total:
            bloques.append({
                "origen": origen,
                "destino_raw": None,
                "duracion_s2": duracion_s2,
                "duracion_destino": None
            })
            break
        # Guardamos el destino en formato crudo (0-7)
        destino_raw = secuencia[fin]
        # Calculamos la duración sobre el valor crudo (no agrupado)
        duracion_destino = 1
        cursor = fin + 1
        while cursor < total and secuencia[cursor] == destino_raw:
            duracion_destino += 1
            cursor += 1
        bloques.append({
            "origen": origen,
            "destino_raw": destino_raw,  # Valor crudo (ej. 3 o 4, no agrupado)
            "duracion_s2": duracion_s2,
            "duracion_destino": duracion_destino  # Duración del bloque crudo
        })
        indice = fin
    return bloques


def resumen_metricas(valores):
    if not valores:
        return {
            "n": 0,
            "promedio": np.nan,
            "mediana": np.nan,
            "desviacion": np.nan,
            "min": np.nan,
            "max": np.nan
        }
    serie = pd.Series(valores)
    return {
        "n": len(valores),
        "promedio": serie.mean(),
        "mediana": serie.median(),
        "desviacion": serie.std(ddof=0) if len(valores) > 1 else 0.0,
        "min": serie.min(),
        "max": serie.max()
    }


def construir_resumen_transiciones(bloques):
    """
    Agrupa por destino (aplicando agrupación solo para etiquetas) y devuelve estadísticas emparejadas espera/llegada.
    
    CORRECCIÓN METODOLÓGICA: Las duraciones se calculan sobre datos crudos (separando S3 de S4).
    La agrupación se aplica solo al agrupar las estadísticas por etiqueta de visualización.
    
    IMPORTANTE: Las duraciones de llegada se mantienen individuales por transición.
    No se suman segmentos consecutivos. Por ejemplo:
    - Si hay transiciones 2→3 (S3) con duraciones [5, 9] y 2→4 (S4) con duraciones [7, 17],
      al agrupar bajo destino=3 (S3/S4), la lista de llegadas será [5, 9, 7, 17],
      y el máximo será 17 (no 9+17=26 ni ningún otro valor sumado).
    """
    datos = defaultdict(lambda: {"espera": [], "llegada": []})
    for bloque in bloques:
        destino_raw = bloque["destino_raw"]
        if destino_raw is None:
            continue
        # Aplicamos agrupación solo para la clave de agrupación
        destino_agrupado = agrupar_fase(destino_raw)
        datos[destino_agrupado]["espera"].append(bloque["duracion_s2"])
        if bloque["duracion_destino"] is not None:
            # Cada duración se añade individualmente; NO se suman
            # Estas duraciones son del bloque crudo (ej. solo S3 o solo S4)
            datos[destino_agrupado]["llegada"].append(bloque["duracion_destino"])
    filas = []
    for destino, mediciones in sorted(datos.items()):
        res_espera = resumen_metricas(mediciones["espera"])
        res_llegada = resumen_metricas(mediciones["llegada"])
        filas.append({
            "Destino": destino,
            "Etiqueta": ETIQUETAS_FASES.get(destino, str(destino)),
            "Transiciones": res_espera["n"],
            "Espera media": res_espera["promedio"],
            "Espera mediana": res_espera["mediana"],
            "Llegada media": res_llegada["promedio"],
            "Llegada mediana": res_llegada["mediana"],
            "Espera min": res_espera["min"],
            "Espera max": res_espera["max"],
            "Llegada min": res_llegada["min"],
            "Llegada max": res_llegada["max"]
        })
    columnas = [
        "Destino", "Etiqueta", "Transiciones",
        "Espera media", "Espera mediana", "Espera min", "Espera max",
        "Llegada media", "Llegada mediana", "Llegada min", "Llegada max"
    ]
    return pd.DataFrame(filas, columns=columnas)

In [30]:
bloques_por_paciente = {}
resumenes_transiciones = {}

for paciente in pacientes:
    bloques = extraer_bloques_s2(hipnogramas[paciente])
    bloques_por_paciente[paciente] = bloques
    resumen = construir_resumen_transiciones(bloques)
    resumenes_transiciones[paciente] = resumen
    print(f"\nPaciente {paciente} transiciones 2 → X: {len(bloques)}")
    display(resumen.style.format(precision=3).hide(axis="index"))

bloques_globales = [bloque for lista in bloques_por_paciente.values() for bloque in lista]
print(f"\nTotal de bloques S2 (todos los pacientes): {len(bloques_globales)}")
resumen_global = construir_resumen_transiciones(bloques_globales)
display(resumen_global.style.format(precision=3).hide(axis="index"))


Paciente AR_2 transiciones 2 → X: 46


Destino,Etiqueta,Transiciones,Espera media,Espera mediana,Espera min,Espera max,Llegada media,Llegada mediana,Llegada min,Llegada max
1,S1,2,4.000,4.000,1,7,1.000,1.000,1,1
3,S3/S4,21,9.000,4.000,1,39,1.619,1.000,1,5
4,REM,7,8.286,8.000,1,14,33.000,34.000,2,72
5,Movimiento/SC,16,14.188,11.500,1,30,1.500,1.000,1,7



Paciente DG_2 transiciones 2 → X: 57


Destino,Etiqueta,Transiciones,Espera media,Espera mediana,Espera min,Espera max,Llegada media,Llegada mediana,Llegada min,Llegada max
0,Vigilia (W),3,10.667,5.000,3,24,1.000,1.000,1,1
1,S1,8,4.000,2.000,1,11,1.125,1.000,1,2
3,S3/S4,26,10.385,2.500,1,59,1.462,1.000,1,4
4,REM,7,3.000,3.000,1,6,7.286,8.000,1,14
5,Movimiento/SC,13,7.923,4.000,2,31,1.462,1.000,1,3



Paciente EL_2 transiciones 2 → X: 52


...

Destino,Etiqueta,Transiciones,Espera media,Espera mediana,Espera min,Espera max,Llegada media,Llegada mediana,Llegada min,Llegada max
0,Vigilia (W),8,4.875,3.000,1,15,4.375,3.500,1,15
1,S1,3,7.000,2.000,1,18,2.333,2.000,1,4
3,S3/S4,15,11.200,8.000,1,40,1.667,1.000,1,6
4,REM,9,4.778,1.000,1,24,13.222,10.000,1,52
5,Movimiento/SC,6,16.500,17.000,5,29,1.000,1.000,1,1



Paciente VC_2 transiciones 2 → X: 37


Destino,Etiqueta,Transiciones,Espera media,Espera mediana,Espera min,Espera max,Llegada media,Llegada mediana,Llegada min,Llegada max
0,Vigilia (W),2,8.000,8.000,3,13,1.500,1.500,1,2
3,S3/S4,14,11.000,9.500,1,31,1.500,1.000,1,4
4,REM,10,6.500,2.000,1,37,19.800,11.000,3,57
5,Movimiento/SC,11,13.909,10.000,2,29,1.000,1.000,1,1



Total de bloques S2 (todos los pacientes): 505


Destino,Etiqueta,Transiciones,Espera media,Espera mediana,Espera min,Espera max,Llegada media,Llegada mediana,Llegada min,Llegada max
0,Vigilia (W),26,6.000,3.000,1,24,2.538,1.500,1,15
1,S1,33,7.091,3.000,1,33,1.394,1.000,1,5
3,S3/S4,236,8.648,3.500,1,77,1.797,1.000,1,17
4,REM,74,6.608,3.000,1,40,19.730,12.000,1,74
5,Movimiento/SC,136,11.404,9.000,1,65,1.250,1.000,1,7


## DataFrame para visualizaciones

Convertimos los bloques por paciente en un DataFrame largo (formato "tidy data") que conserve:
- **Paciente**: identificador del paciente
- **Destino**: código numérico del destino agrupado (3=S3/S4, 4=REM, etc.)
- **Etiqueta destino**: etiqueta legible del destino (S3/S4, REM, etc.)
- **Duración S2**: tiempo de espera en S2 antes de la transición
- **Duración destino**: tiempo de llegada en la fase destino inmediatamente después

In [31]:
def construir_dataframe_bloques(bloques_por_paciente):
    """
    Convierte los bloques extraídos en un DataFrame estructurado para visualizaciones.
    
    CORRECCIÓN METODOLÓGICA: Calcula duraciones usando datos crudos (separando S3 de S4).
    La agrupación se aplica solo a las etiquetas de visualización, no a las duraciones.
    
    Esta función toma el diccionario de bloques por paciente (que contiene listas de dicts
    con metadatos de cada transición S2→destino) y los convierte en un formato tabular
    que facilita:
    - Generar boxplots con Plotly
    - Filtrar por paciente o destino
    
    Cada fila representa una transición individual 2→X con sus métricas de espera y llegada.
    Las duraciones de llegada son del bloque crudo (ej. solo S3 o solo S4, no sumadas).
    """
    filas = []
    for paciente, bloques in bloques_por_paciente.items():
        for bloque in bloques:
            destino_raw = bloque["destino_raw"]
            if destino_raw is None:
                continue  # Saltamos bloques que terminan en S2 (sin destino)
            
            # Aplicamos agrupación solo para la etiqueta de visualización
            destino_agrupado = agrupar_fase(destino_raw)
            
            filas.append({
                "Paciente": paciente,
                "Destino Raw": destino_raw,  # Valor crudo (0-7) para referencia
                "Destino": destino_agrupado,  # Código agrupado (3=S3/S4, 4=REM, etc.)
                "Etiqueta destino": ETIQUETAS_FASES.get(destino_agrupado, str(destino_agrupado)),
                "Duración S2": bloque["duracion_s2"],  # Espera en S2
                "Duración destino": bloque["duracion_destino"]  # Llegada al destino (crudo, no sumado)
            })
    return pd.DataFrame(filas)


# Construimos el DataFrame con todas las transiciones de todos los pacientes
df_bloques = construir_dataframe_bloques(bloques_por_paciente)
print(f"Total de transiciones en el DataFrame: {len(df_bloques)}")
df_bloques

Total de transiciones en el DataFrame: 505


,Paciente,Destino Raw,Destino,Etiqueta destino,Duración S2,Duración destino
0,AR_2,3,3,S3/S4,12,1
1,AR_2,3,3,S3/S4,1,1
2,AR_2,3,3,S3/S4,5,2
3,AR_2,3,3,S3/S4,1,1
4,AR_2,3,3,S3/S4,8,1
...,...,...,...,...,...,...
500,VC_2,3,3,S3/S4,1,2
501,VC_2,3,3,S3/S4,3,1
502,VC_2,3,3,S3/S4,1,2
503,VC_2,5,4,REM,2,12


In [39]:
# Calcular matriz de transición SIN diagonal para enriquecer el boxplot
from collections import Counter

def extraer_bigramas(secuencia):
    """Extrae todos los bigramas (pares consecutivos) de una secuencia."""
    return [(secuencia[i], secuencia[i+1]) for i in range(len(secuencia)-1)]

def construir_matriz_transicion(bigramas):
    """Construye la matriz de transición a partir de bigramas."""
    fases_unicas = sorted(set([fase for bigrama in bigramas for fase in bigrama]))
    matriz_absoluta = pd.DataFrame(
        0, 
        index=fases_unicas, 
        columns=fases_unicas, 
        dtype=int
    )
    contador = Counter(bigramas)
    for (desde, hacia), count in contador.items():
        if desde in matriz_absoluta.index and hacia in matriz_absoluta.columns:
            matriz_absoluta.loc[desde, hacia] = count
    suma_por_fila = matriz_absoluta.sum(axis=1)
    matriz_probabilidades = matriz_absoluta.div(suma_por_fila, axis=0).fillna(0)
    return matriz_probabilidades

def construir_matriz_sin_diagonal(matriz_probabilidades):
    """Construye una versión de la matriz donde la diagonal es cero y se re-normaliza por fila."""
    matriz_sin_diag = matriz_probabilidades.copy()
    for fase in matriz_sin_diag.index:
        if fase in matriz_sin_diag.columns:
            matriz_sin_diag.loc[fase, fase] = 0
    suma_por_fila = matriz_sin_diag.sum(axis=1)
    matriz_sin_diag = matriz_sin_diag.div(suma_por_fila, axis=0).fillna(0)
    return matriz_sin_diag

# Agrupar todos los hipnogramas y calcular bigramas
hipnogramas_agrupados = {}
for paciente, secuencia in hipnogramas.items():
    secuencia_agrupada = remapear_secuencia(secuencia)
    hipnogramas_agrupados[paciente] = secuencia_agrupada

# Extraer todos los bigramas de todos los pacientes
todos_los_bigramas = []
for secuencia in hipnogramas_agrupados.values():
    todos_los_bigramas.extend(extraer_bigramas(secuencia))

# Construir matriz de transición
matriz_prob = construir_matriz_transicion(todos_los_bigramas)
matriz_sin_diag = construir_matriz_sin_diagonal(matriz_prob)

# Extraer probabilidades de transición desde S2 (fase 2)
fase_s2 = 2
probabilidades_s2 = {}
if fase_s2 in matriz_sin_diag.index:
    for fase_destino in matriz_sin_diag.columns:
        if fase_destino != fase_s2:  # Solo destinos diferentes de S2
            prob = matriz_sin_diag.loc[fase_s2, fase_destino]
            etiqueta_destino = ETIQUETAS_FASES.get(fase_destino, f"Fase {fase_destino}")
            probabilidades_s2[etiqueta_destino] = prob

print("Probabilidades de transición desde S2 (matriz sin diagonal, renormalizada):")
for etiqueta, prob in sorted(probabilidades_s2.items(), key=lambda x: x[1], reverse=True):
    print(f"  S2 → {etiqueta}: {prob:.3f} ({prob*100:.1f}%)")


Probabilidades de transición desde S2 (matriz sin diagonal, renormalizada):
  S2 → S3/S4: 0.467 (46.7%)
  S2 → Movimiento/SC: 0.269 (26.9%)
  S2 → REM: 0.147 (14.7%)
  S2 → S1: 0.065 (6.5%)
  S2 → Vigilia (W): 0.051 (5.1%)


## Boxplots emparejados (espera vs llegada)

1. **Por paciente**: cada caja resume todas las esperas (o llegadas) observadas en ese individuo.
2. **Global**: misma métrica pero sin desglosar por paciente.


In [40]:
# Enriquecer el boxplot con probabilidades de transición (SIN diagonal)
# Esta celda modifica el gráfico anterior para agregar las probabilidades debajo de cada destino

if not df_bloques.empty and 'fig_destinos_combinado' in locals():
    # Obtener el rango del eje Y para posicionar las anotaciones
    y_min = df_long["Duración"].min()
    y_max = df_long["Duración"].max()
    y_pos_anotacion = y_min - (y_max - y_min) * 0.15  # 15% debajo del mínimo
    
    # Agregar anotación para cada destino
    for idx, destino in enumerate(orden_destinos):
        if destino in probabilidades_s2:
            prob = probabilidades_s2[destino]
            texto_anotacion = f"P = {prob:.3f}<br>({prob*100:.1f}%)"
            
            fig_destinos_combinado.add_annotation(
                x=idx,
                y=y_pos_anotacion,
                text=texto_anotacion,
                showarrow=False,
                font=dict(color="white", size=11),
                bgcolor="rgba(0,0,0,0.7)",
                bordercolor="white",
                borderwidth=1,
                xref="x",
                yref="y",
                xanchor="center",
                yanchor="top"
            )
    
    # Ajustar el rango del eje Y para que las anotaciones sean visibles
    fig_destinos_combinado.update_yaxes(range=[y_pos_anotacion - (y_max - y_min) * 0.1, y_max * 1.05])
    
    # Actualizar título
    fig_destinos_combinado.update_layout(
        title="Comparación de espera en S2 vs llegada por destino<br><sub>Probabilidades de transición (sin diagonal) mostradas debajo de cada destino</sub>"
    )
    
    fig_destinos_combinado.show()
else:
    print("Ejecuta primero la celda anterior del boxplot.")


**Probabilidades Re-normalizadas**: Probabilidades de la matriz sin diagonal. Esto es clave porque nos interesa la probabilidad de elegir un destino dado que ya se decidió salir de S2.

**Ejemplo**: S2 → S3/S4 tiene una probabilidad del 46.7% (0.467), lo cual es consistente con las matrices de transición.

### Conclusiones Biológicas del Boxplot Enriquecido

Al observar el gráfico combinado (Boxplot de Duración + Probabilidad de Transición), surgen patrones muy interesantes que fortalecen la tesis:

#### A. La "Inversión" Duración-Probabilidad en SWS vs REM

**Observación**:
- **S2 → S3/S4 (Sueño Profundo)**: Es el destino más probable (46.7%), pero tiene una mediana de duración relativamente baja (~3 épocas) y una distribución compacta.
- **S2 → REM**: Es menos probable (14.7%), pero su mediana es similar o ligeramente mayor, con una distribución mucho más dispersa (más outliers hacia arriba).

**Conclusión**: La transición hacia el sueño profundo es un proceso más "determinista" y rápido. Cuando el cerebro necesita recuperación física, la Fase 2 actúa como un simple trámite corto. En cambio, la transición a REM parece requerir condiciones más específicas o una "preparación" más variable, lo que explica la mayor dispersión en los tiempos de espera.

#### B. La Inestabilidad de la Vigilia y el Movimiento

**Observación**:
- **S2 → Vigilia (W)**: Probabilidad muy baja (5.1%) y duraciones muy cortas (mediana ~2-3 épocas).
- **S2 → Movimiento**: Probabilidad media (26.9%).

**Conclusión**: Las salidas hacia Vigilia o Movimiento son eventos abruptos. No hay una "preparación" larga en S2; simplemente ocurren (probablemente por estímulos externos o micro-despertares). Esto contrasta con las transiciones fisiológicas a SWS o REM, que muestran una estructura temporal más organizada.

#### C. Validación de la Hipótesis de "Preparación"

El gráfico demuestra que el tiempo que se pasa en S2 no es aleatorio.

- Si se va hacia S3/S4 (lo más común), el tiempo es corto y consistente.
- Si se va hacia REM, el tiempo es más variable.

**Conclusión**: Esto sugiere que S2 tiene sub-estados funcionales distintos: un "S2 rápido" que lleva al fondo (SWS) y un "S2 inestable/largo" que a veces logra abrir la puerta al sueño (REM).

In [32]:
if df_bloques.empty:
    print("No hay bloques suficientes para construir boxplots.")
else:
    orden_pacientes = sorted(df_bloques["Paciente"].unique())

    fig_espera_pacientes = px.box(
        df_bloques,
        x="Paciente",
        y="Duración S2",
        color="Paciente",
        points="outliers",
        category_orders={"Paciente": orden_pacientes},
        title="Distribución de esperas en S2 por paciente",
        template="plotly_dark"
    )
    fig_espera_pacientes.update_layout(
        xaxis_title="Paciente",
        yaxis_title="Duración en S2 (épocas)",
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend_title_text="Paciente"
    )
    fig_espera_pacientes.show()

    fig_llegada_pacientes = px.box(
        df_bloques,
        x="Paciente",
        y="Duración destino",
        color="Paciente",
        points="outliers",
        category_orders={"Paciente": orden_pacientes},
        title="Distribución de llegadas (fase destino) por paciente",
        template="plotly_dark"
    )
    fig_llegada_pacientes.update_layout(
        xaxis_title="Paciente",
        yaxis_title="Duración destino (épocas)",
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend_title_text="Paciente"
    )
    fig_llegada_pacientes.show()

    fig_espera_global = px.box(
        df_bloques,
        y="Duración S2",
        points="outliers",
        title="Distribución global de esperas en S2",
        template="plotly_dark"
    )
    fig_espera_global.update_layout(
        xaxis_title="",
        yaxis_title="Duración en S2 (épocas)",
        showlegend=False,
        paper_bgcolor="black",
        plot_bgcolor="black"
    )
    fig_espera_global.show()

    fig_llegada_global = px.box(
        df_bloques,
        y="Duración destino",
        points="outliers",
        title="Distribución global de llegadas",
        template="plotly_dark"
    )
    fig_llegada_global.update_layout(
        xaxis_title="",
        yaxis_title="Duración destino (épocas)",
        showlegend=False,
        paper_bgcolor="black",
        plot_bgcolor="black"
    )
    fig_llegada_global.show()


## Boxplot combinado por fase destino

En un mismo gráfico mostramos, para cada destino distinto de S2, dos cajas adyacentes: la espera en S2 y la duración del destino inmediatamente posterior.


In [ ]:
if df_bloques.empty:
    print("Sin datos para agrupar por destino.")
else:
    etiqueta_s2 = ETIQUETAS_FASES[2]
    df_destinos = df_bloques[df_bloques["Etiqueta destino"] != etiqueta_s2].copy()
    if df_destinos.empty:
        print("Solo se detectaron transiciones 2→2; no hay destinos adicionales que mostrar.")
    else:
        print("Conteo de destinos presentes en los boxplots combinados:")
        display(df_destinos["Etiqueta destino"].value_counts().rename("Segmentos"))

        registros = []
        for _, fila in df_destinos.iterrows():
            if pd.notnull(fila["Duración S2"]):
                registros.append({
                    "Etiqueta destino": fila["Etiqueta destino"],
                    "Duración": fila["Duración S2"],
                    "Tipo": "Espera en S2"
                })
            if pd.notnull(fila["Duración destino"]):
                registros.append({
                    "Etiqueta destino": fila["Etiqueta destino"],
                    "Duración": fila["Duración destino"],
                    "Tipo": "Llegada destino"
                })
        df_long = pd.DataFrame(registros)
        orden_destinos = [et for et in ORDEN_DESTINOS if et in df_long["Etiqueta destino"].unique()]
        fig_destinos_combinado = px.box(
            df_long,
            x="Etiqueta destino",
            y="Duración",
            color="Tipo",
            points="outliers",
            title="Comparación de espera en S2 vs llegada por destino",
            template="plotly_dark",
            category_orders={
                "Etiqueta destino": orden_destinos,
                "Tipo": ["Espera en S2", "Llegada destino"]
            }
        )
        fig_destinos_combinado.update_layout(
            xaxis_title="Destino",
            yaxis_title="Duración (épocas)",
            paper_bgcolor="black",
            plot_bgcolor="black",
            legend_title_text="Métrica"
        )
        fig_destinos_combinado.show()

Conteo de destinos presentes en los boxplots combinados:


Etiqueta destino
S3/S4            236
Movimiento/SC    136
REM               74
S1                33
Vigilia (W)       26
Name: Segmentos, dtype: int64

### Agregar la probabilidad de transicion -> a los Boxplots -> Abajo de de cada fase

Bigramas - Boxplots

In [34]:
def calcular_frecuencias_loglog(df, columna_duracion="Duración S2", grupo=None):
    """
    Calcula la frecuencia de cada duración para el gráfico Log-Log.
    """
    # Filtrar nulos
    datos = df.dropna(subset=[columna_duracion])
    
    if grupo:
        # Frecuencias por grupo (ej. Paciente)
        conteo = datos.groupby([grupo, columna_duracion]).size().reset_index(name="Frecuencia")
    else:
        # Frecuencia global
        conteo = datos[columna_duracion].value_counts().reset_index()
        conteo.columns = [columna_duracion, "Frecuencia"]
        conteo["Etiqueta"] = "Global"
    
    return conteo

# 1. Calcular frecuencias globales
frec_global = calcular_frecuencias_loglog(df_bloques, "Duración S2")

# 2. Calcular frecuencias por paciente (opcional, para ver si todos siguen la ley)
frec_pacientes = calcular_frecuencias_loglog(df_bloques, "Duración S2", grupo="Paciente")

print("Datos de frecuencia listos para graficar.")
display(frec_global.head())

Datos de frecuencia listos para graficar.


,Duración S2,Frecuencia,Etiqueta
0,1,122,Global
1,2,59,Global
2,3,37,Global
3,5,30,Global
4,4,29,Global


In [35]:
# Crear la figura
fig_loglog = go.Figure()

# 1. Agregar los datos por Paciente (puntos pequeños y tenues de fondo)
pacientes_unicos = frec_pacientes["Paciente"].unique()
for p in pacientes_unicos:
    datos_p = frec_pacientes[frec_pacientes["Paciente"] == p]
    fig_loglog.add_trace(go.Scatter(
        x=datos_p["Duración S2"],
        y=datos_p["Frecuencia"],
        mode='markers',
        name=p,
        marker=dict(size=5, opacity=0.35),
        visible='legendonly'  # Ocultos por defecto para no saturar
    ))

# 2. Agregar los datos Globales (puntos grandes y destacados)
fig_loglog.add_trace(go.Scatter(
    x=frec_global["Duración S2"],
    y=frec_global["Frecuencia"],
    mode='markers',
    name='Global (Todos)',
    marker=dict(size=10, color='white', line=dict(width=2, color='royalblue'))
))

# 2b. Ajuste lineal en el espacio log-log
validos = frec_global[(frec_global["Duración S2"] > 0) & (frec_global["Frecuencia"] > 0)].copy()
log_x = np.log10(validos["Duración S2"].values)
log_y = np.log10(validos["Frecuencia"].values)
pendiente, intercepto = np.polyfit(log_x, log_y, 1)
log_y_pred = pendiente * log_x + intercepto
ss_res = np.sum((log_y - log_y_pred) ** 2)
ss_tot = np.sum((log_y - log_y.mean()) ** 2)
r2 = 1 - ss_res / ss_tot if len(log_y) > 1 else np.nan

orden = np.argsort(validos["Duración S2"].values)
x_line = validos["Duración S2"].values[orden]
y_line = 10 ** (intercepto + pendiente * np.log10(x_line))
fig_loglog.add_trace(go.Scatter(
    x=x_line,
    y=y_line,
    mode='lines',
    name='Regresión lineal',
    line=dict(color='white', width=2)
))

# 3. Configurar escalas Logarítmicas (La clave de la Ley de Potencia)
x_ticks = [1, 2, 4, 8, 16, 32, 64]
y_ticks = [1, 2, 4, 8, 16, 32, 64, 128]

fig_loglog.update_layout(
    title="Ley de Potencia: Distribución de Duraciones de Fase 2 (Log-Log)",
    xaxis=dict(
        title="Duración del segmento (épocas)",
        type="log"
    ),
    yaxis=dict(
        title="Frecuencia (cuántas veces ocurre)",
        type="log"
    ),
    template="plotly_dark",
    height=600,
    legend=dict(title="Datos")
)

fig_loglog.update_xaxes(
    tickmode="array",
    tickvals=x_ticks,
    ticktext=[str(v) for v in x_ticks]
)
fig_loglog.update_yaxes(
    tickmode="array",
    tickvals=y_ticks,
    ticktext=[str(v) for v in y_ticks]
)

fig_loglog.show()

print("Métricas ley de potencia (datos globales, escala log10):")
print(f"  Pendiente: {pendiente:.3f}")
print(f"  Intercepto: {intercepto:.3f}")
print(f"  R^2: {r2:.3f}")

Métricas ley de potencia (datos globales, escala log10):
  Pendiente: -1.300
  Intercepto: 2.296
  R^2: 0.882


In [36]:
def detectar_ciclos_rem(secuencia, fase_rem=4, umbral_separacion_rem=20, umbral_ciclo=60, epocas_por_minuto=2):
    """
    Detecta ciclos de sueño de forma robusta agrupando REM.
    """
    # Busca la fase indicada (por defecto 4, que será REM tras remapear)
    indices_rem = [i for i, x in enumerate(secuencia) if x == fase_rem]
    
    if not indices_rem:
        return []

    # Consolidar bloques REM
    periodos_rem = []
    if indices_rem:
        inicio_actual = indices_rem[0]
        fin_actual = indices_rem[0]
        
        for i in range(1, len(indices_rem)):
            distancia = indices_rem[i] - indices_rem[i-1]
            if distancia <= (umbral_separacion_rem * epocas_por_minuto):
                fin_actual = indices_rem[i]
            else:
                periodos_rem.append((inicio_actual, fin_actual))
                inicio_actual = indices_rem[i]
                fin_actual = indices_rem[i]
        periodos_rem.append((inicio_actual, fin_actual))

    # Definir ciclos
    ciclos = []
    inicio_ciclo = 0
    for p_inicio, p_fin in periodos_rem:
        ciclos.append((inicio_ciclo, p_fin))
        inicio_ciclo = p_fin + 1
        
    if inicio_ciclo < len(secuencia):
        ciclos.append((inicio_ciclo, len(secuencia) - 1))

    return ciclos

def graficar_hipnograma_ciclos(paciente, secuencia_cruda):
    """
    Genera un hipnograma estilo clínico.
    IMPORTANTE: Convierte la secuencia cruda al formato visual (0-5) antes de graficar.
    """
    # 1. REMAPEO LOCAL para visualización correcta
    # Usamos la función agrupar_fase que ya definiste arriba
    secuencia_vis = [agrupar_fase(x) for x in secuencia_cruda]
    
    # 2. Preparar datos
    x = [i / 120 for i in range(len(secuencia_vis))] 
    y = secuencia_vis
    
    # 3. Detectar ciclos usando la secuencia YA REMAPEADA (donde REM=4)
    ciclos = detectar_ciclos_rem(secuencia_vis, fase_rem=4)
    
    fig = go.Figure()
    
    # Sombrear ciclos
    colores_ciclo = ["rgba(200, 200, 200, 0.1)", "rgba(100, 100, 255, 0.1)"]
    for i, (inicio, fin) in enumerate(ciclos):
        fig.add_vrect(
            x0=inicio/120, x1=fin/120,
            fillcolor=colores_ciclo[i % 2], layer="below", line_width=0,
            annotation_text=f"Ciclo {i+1}", annotation_position="top left"
        )

    # Trazar hipnograma
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        line=dict(color='white', width=2, shape='hv'),
        name='Hipnograma'
    ))
    
    # Resaltar REM (Fase 4 en el mapa visual)
    rem_x = [i/120 for i, f in enumerate(secuencia_vis) if f == 4]
    rem_y = [4] * len(rem_x)
    if rem_x:
         fig.add_trace(go.Scatter(
            x=rem_x, y=rem_y,
            mode='markers',
            marker=dict(symbol="square", color="blue", size=5, opacity=0.5),
            name='REM',
            hoverinfo='skip'
        ))

    fig.update_layout(
        title=f"Estructura de Ciclos de Sueño - Paciente {paciente}",
        xaxis=dict(title="Tiempo de sueño (horas)", showgrid=False),
        yaxis=dict(
            title="Fase",
            tickvals=[0, 1, 2, 3, 4, 5],
            ticktext=["W", "N1", "N2", "SWS", "REM", "Mov/SC"], # Actualizado
            autorange="reversed",
            gridcolor="rgba(255,255,255,0.1)"
        ),
        template="plotly_dark",
        height=500,
        showlegend=False
    )
    
    fig.show()

# EJEMPLO DE USO
paciente_ejemplo = "SS_2"
if paciente_ejemplo in hipnogramas:
    # Pasamos el hipnograma crudo, la función se encarga de mapearlo para la foto
    graficar_hipnograma_ciclos(paciente_ejemplo, hipnogramas[paciente_ejemplo])
else:
    print("Carga los datos primero.")

In [37]:
# 1. Función para aislar el Ciclo 1
def obtener_ciclo_1(secuencia, fase_rem=4):
    """
    Retorna la secuencia recortada desde el inicio hasta el final del primer periodo REM.
    """
    try:
        # Encontrar dónde empieza el primer REM
        inicio_rem = secuencia.index(fase_rem)
        
        # Buscar dónde termina ese bloque REM (avanzar mientras siga siendo REM)
        fin_rem = inicio_rem
        while fin_rem < len(secuencia) and secuencia[fin_rem] == fase_rem:
            fin_rem += 1
            
        # Devolver el recorte
        return secuencia[:fin_rem]
    except ValueError:
        # Si no hay REM, devolvemos lista vacía (o toda la secuencia, según criterio. Aquí ignoramos)
        return []

# 2. Extraer duraciones de N2 SOLO del Ciclo 1
duraciones_c1 = []

for paciente, secuencia in hipnogramas.items(): # Usamos el diccionario 'hipnogramas' que ya tienes cargado
    seq_c1 = obtener_ciclo_1(secuencia, fase_rem=4) # 4 es REM en tu mapa unificado
    
    if seq_c1:
        bloques = extraer_bloques_s2(seq_c1)
        # Nos interesan las duraciones de espera en S2
        duraciones = [b["duracion_s2"] for b in bloques]
        duraciones_c1.extend(duraciones)

print(f"Total de bloques S2 analizados en Ciclo 1: {len(duraciones_c1)}")

# 3. Calcular Estadísticas Clave
if duraciones_c1:
    moda_val = stats.mode(duraciones_c1, keepdims=True)[0][0]
    media_val = np.mean(duraciones_c1)
    
    # Preparar datos para Plotly
    df_c1 = pd.DataFrame(duraciones_c1, columns=["Duracion"])
    conteo_c1 = df_c1["Duracion"].value_counts().sort_index()

    # 4. Crear la Gráfica
    fig = go.Figure()

    # Histograma (Barras)
    fig.add_trace(go.Bar(
        x=conteo_c1.index, 
        y=conteo_c1.values,
        name="Frecuencia",
        marker_color='#AB63FA', # Un color morado/lila para distinguir del azul global
        opacity=0.7
    ))

    # Línea de la Moda (Lo más típico)
    fig.add_vline(x=moda_val, line_width=3, line_dash="dash", line_color="red", 
                  annotation_text=f"Moda: {moda_val}", annotation_position="top right")

    # Línea de la Media
    fig.add_vline(x=media_val, line_width=3, line_dash="dash", line_color="cyan", 
                  annotation_text=f"Media: {media_val:.1f}", annotation_position="top right")

    # Layout Estilizado
    fig.update_layout(
        title="<b>Dinámica del Primer Ciclo:</b> Distribución de duraciones de N2",
        xaxis=dict(title="Duración del segmento (épocas)", range=[0, 30]), # Limitamos a 30 para ver bien el inicio
        yaxis=dict(title="Frecuencia"),
        template="plotly_dark",
        bargap=0.1,
        height=600,
        showlegend=False
    )

    fig.show()
    
    print(f"RESULTADO CLAVE: La duración más común (Moda) en el primer ciclo es {moda_val} épocas.")
    print(f"El promedio es {media_val:.2f} épocas.")
else:
    print("No se encontraron datos para el Ciclo 1.")

Total de bloques S2 analizados en Ciclo 1: 30


RESULTADO CLAVE: La duración más común (Moda) en el primer ciclo es 2 épocas.
El promedio es 6.60 épocas.
